# ImageDataGenerator

- Rather than using re-scaling and augmentation layer we can do same thing in better way..

In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
from IPython.display import HTML

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## I will split data 

- `split_folders` model will help to divide the folders

here we divide data into 70% Training,10% validation,20% Testing

In [2]:
import splitfolders 
splitfolders.ratio('PlantVillage', output="dataset", seed=1337, ratio=(.7, 0.1,0.2)) 

Copying files: 2152 files [00:20, 105.74 files/s]


It had make dataset new folder with train,val,test

In [21]:
IMAGE_SIZE=256
CHANNELS=3

train_datagen=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    
    
)


train_generator=train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse',
    
)

Found 1506 images belonging to 3 classes.


In [7]:
for image_batch,label_batch in trian_generator:
    print(image_batch.shape)
    print(label_batch)
    break

(32, 256, 256, 3)
[0. 1. 2. 1. 1. 0. 0. 1. 2. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 2. 0. 0. 0.
 1. 1. 1. 1. 2. 1. 0. 2.]


Same for validation

In [8]:
val_datagen=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    
    
)


val_generator=train_datagen.flow_from_directory(
    'dataset/val',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse',
    
)

Found 215 images belonging to 3 classes.


for test

In [9]:
test_datagen=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    
    
)


test_generator=train_datagen.flow_from_directory(
    'dataset/test',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse',
    
)

Found 431 images belonging to 3 classes.


#### Model Building

In [14]:
input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes=3
model=models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 30, 30, 64)        0

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

steps_per_epoch value is depend on number of images and batch_size

In [17]:
1506/32

47.0625

Same as above for validation_data

In [25]:
215/32

6.71875

In [27]:
EPOCHS=2

model.fit(
    train_generator,
    steps_per_epoch=47,
    batch_size=32,
    validation_data=val_generator,
    validation_steps=6,
    verbose=1,
    epochs=EPOCHS
)

Epoch 1/2
47/47 [==============================] - 67s 1s/step - loss: 0.1254 - accuracy: 0.9545 - val_loss: 0.2905 - val_accuracy: 0.9010
Epoch 2/2
47/47 [==============================] - 68s 1s/step - loss: 0.1233 - accuracy: 0.9491 - val_loss: 0.1766 - val_accuracy: 0.9219


## Save Model
-  Here we save model in `model.h5`, rather than saving model in directory we save model in 1 single file.

In [28]:
model.save('potato_imageDataGenerator.h5')

C:\Users\hp\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
